# Tangelo hands-on: VQE framework

## Before you jump in

Before tackling this hands-on, we suggest to have a look at:

- the tutorial on the basics of the linq module
- the tutorial on the basics of quantum chemistry modeling

This hands-on covers some material described in the [following tutorials](https://github.com/goodchemistryco/Tangelo-Examples/tree/main/examples/variational_methods).

You can use the cells below to install or import useful packages. We recommend to install qulacs and qiskit.

In [ ]:
import numpy as np
import time

If you wish to install anything in your jupyter environment while working on this notebook, including Tangelo, you can go through your terminal and reload the upyter kernel or run `pip` commands directly from this notebook using `!`

In [ ]:
# Uncomment and change package name to install it in the current environment
# !pip install tangelo-gc --quiet
# !pip install qulacs --quiet
# !pip install qiskit --quiet

## Level 1: VQE basics

Let's define a molecule in second quantization, for the sake of exploring the variational framework. Here we define a molecule explicitly, specifying a geometry, charge, spin and basis set, but you could also quickly just import other existing ones in our ["molecule library"](https://github.com/goodchemistryco/Tangelo/blob/main/tangelo/molecule_library.py) if you want to swap between other examples.

In [ ]:
# Define H2 molecule in minimal basis set
from tangelo import SecondQuantizedMolecule

xyz_H2 = [("H", (0., 0., 0.)), ("H", (0., 0., 0.7414))]
mol_H2_sto3g = SecondQuantizedMolecule(xyz_H2, q=0, spin=0, basis="sto-3g")

mol = mol_H2_sto3g

### 1a)  Computing the ground state energy with VQE

Write a code cell that instantiates a `VQESolver` object and computes the ground state energy of our molecule. You can use the default options. You should find that the result for the optimal geometry of H2 in sto-3g basis is roughly -1.1372702 Ha.

In [ ]:
# TODO: Instantiate VQE Solver, compute ground state energy with VQE


Many Can you find what qubit mapping, ansatz and classical optimizer were used in your code ?

In [ ]:
# TODO : print that information


### 1b) A closer look at what happened

Please print:

- the accuracy of the previous computation (difference between your optimal VQE energy and the result coming from the `FCISolver`)
- the optimal circuit, and the optimal parameters
- the qubit operator that encoded your molecule 

In [ ]:
# TODO compute and print the information

## Level 2: Impact of some of the options in VQE

A number of options are available in the VQE framework. Let's see how these choices impact the calculations.

### 2a) Qubit mappings

Our VQE framework supports a number of qubit mappings, such as Jordan-Wigner (`'jw'`), Bravyi-Kitaev (`'bk'`) or Symmetry-Conserving Bravyi-Kitaev (`'bk'`). We also support more uncommon ones, such as JKMN or HCB, and probably more in the future !

Can you instantiate `VQESolver` objects using these different qubit mappings, and see how they impact the complexity of the algorithm ?

In [ ]:
for qb_mapping in ["JW", "BK", "SCBK", "JKMN"]:
    # TODO: Try different qubit mappings. How do they impact to the computational resource requirements ?

### 2b) Compute backends

Tangelo is backend-agnostic. That means that you do not have to rewrite your code if you want to run it on a different simulator or quantum device, and it takes little effort to swap or integrate with a new backend overall.

There's a lot of quantum simulators out there, with different tradeoffs between speed, accuracy, maximal circuit size or ability to simulate noise. Likewise, there's a growing number of quantum devices out there, based on different technologies.

Let's revisit H2 in a bigger basis set, stick with the Jordan-Wigner qubit mapping, and see what impact the choice of backend has on our experience.

In [ ]:
from tangelo.molecule_library import mol_H2_321g

mol = mol_H2_321g

In [ ]:
# TODO: Without simulating the algorithm, get an idea of the amount of resources this problem would require

Now let's see what happens when you try to run VQE with the `'jw'` mapping but with the following backends. It may take some time depending of your machine, and how optimally these packages were installed (If you've waited more than a few minutes, you can try to revert to H2 in sto-3g basis)

In [ ]:
for b in ["qulacs", "cirq", "qiskit"]:
    
    # TODO : Create the dictionary of options to build our VQE object with the desired backends
    vqe_options = # xxx

    # Start timer
    t_start = time.time()
    
    # Run VQE algorithm
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    opt_energy = vqe_solver.simulate()
    
    # Stop timer, print elapsed time
    t_stop = time.time()
    print(f"{b:10} :: {t_stop - t_start:3.3f} s \t (energy = {opt_energy})")

Roughly, what is the speedup between the best- and poorest-performing backends on your machine ?

### 2c) Ansatz 

The ansatz refers to the variational circuit used to estimate the energy of the molecule, and features a number of variational parameters that need to be optimized classically. Tangelo features a growing collection of built-in ansatze, and also gives you the opportunity to define your own. Its choice is overall pretty arbitrary.

Can you change the following code, and try a few of the built-in ansatze (UCCSD, UpCCGSC, HEA, UCC1...) ?
What do you observe ?

In [ ]:
from tangelo.algorithms import BuiltInAnsatze as Ansatze
from tangelo.algorithms import VQESolver

# TODO : change vqe_options to try a few different ansatze
vqe_options = {"molecule": mol}

# Start timer
t_start = time.time()

# Run VQE algorithm
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(vqe_solver.get_resources())
opt_energy = vqe_solver.simulate()

# Stop timer, print elapsed time
t_stop = time.time()
print(f"{ans.name:10} :: {t_stop - t_start:3.3f} s \t (energy = {opt_energy})")

## Boss fight: H2O

You want to compute the ground state energy of H2O in sto-3g basis set using VQE: this problem is more challenging than the previous ones. Use all you can find in the source code, documentation and tutorials to give it a go !

- More accuracy is better. Can you reach chemical accuracy (~0.001 Ha) ?
- Research time is important: how much can you reduce time-to-solution with your code ?
- Quantum computers and simulators are limited: how much can you reduce computational requirements ?

Think about all the degrees of freedom available to you, and try to explore a few of the most promising avenues you can think of.

In [ ]:
from tangelo.molecule_library import mol_H2O_sto3g
mol = mol_H2O_sto3g

fci_h2o = FCISolver(mol)
opt_e = fci_h2o.simulate()
print(f"H2O FCI energy = {opt_e}")

In [ ]:
# TODO: fight !

# Final words

After all these examples, what observations can you make about the VQE algorithm and what seems to be important in order to apply it successfully to your research ? What were the challenges you've encountered ?

We've only grazed the surface here, but there's more to VQE and its variations. If it is relavant to you, take a look at the [following tutorials](https://github.com/goodchemistryco/Tangelo-Examples/tree/main/examples/variational_methods). Did you know that VQE can also compute [excited states](https://github.com/goodchemistryco/Tangelo-Examples/blob/main/examples/chemistry/excited_states.ipynb)? Keep an eye out, maybe 'watch' the Github repo: the community keeps coming up with new innovations. Maybe we'll owe the next one to you :)

What will you do with Tangelo ?